# Introduction

This is an introduction to `NCA.jl`, a software for noncompartmental analysis (NCA).
In this tutorial we will show how to use `NCA.jl` to analysis data.

## Installation

Currently, `NCA.jl` is a submodule in `Pumas.jl`, so you only need to install
`Pumas.jl`, and everything will be ready to go.

## Getting Started

To load the package, use
# NCA Tutorial
### Yingbo Ma
### February 12, 2019

In [ ]:
using Pumas.NCA

First, let's load the example NCA data inside `Pumas.jl`. This data have $24$
individuals, and each of them has $16$ data points.

In [ ]:
using Pumas, CSV

file = Pumas.example_nmtran_data("nca_test_data/dapa_IV")
data = CSV.read(file)

here is what the dataset looks like

In [ ]:
first(data, 6) # take first 6 rows

# Efficient Computation of Multiple NCA Diagnostics

## AUC and AUMC

We can compute the area under the curve (AUC) from the first observation time
to infinity. Below we are accessing the concentration and corresponding time
array for the first individual. By default, the `auc` function computes the AUC
from initial time to infinity (AUCinf).

In [ ]:
NCA.auc(data[:CObs][1:16], data[:TIME][1:16])

In [ ]:
NCA.auc(data[:CObs][1:16], data[:TIME][1:16], method=:linuplogdown)

the keyword argument `method` can be `:linear`, `:linuplogdown`, or `:linlog`,
and it defaults to `:linear`. This is a simple interface, however it is not
efficient if you want to compute many quantities. The recommended way is to
create an `NCASubject` or an `NCAPopulation` object first and then call the
respective NCA diagnostic on the data object. To parse data to an
`NCAPopulation` object one can call the `parse_ncadata` function and give
column names of `id`, `time`, `conc` (concentration), `amt` (dosage),
`formulation`, `iv` (IV bolus name). Note that, by default, the lower limit of
quantization (LLQ) is $0$, and concentrations that are below LLQ (BLQ) are
dropped. Also, we can add units by providing `timeu`, `concu`, and `amtu`.

In [ ]:
timeu = u"hr"
concu = u"mg/L"
amtu  = u"mg"
pop = parse_ncadata(data, id=:ID, time=:TIME, conc=:CObs, amt=:AMT_IV, formulation=:Formulation, iv="IV",
  llq=0concu, timeu=timeu, concu=concu, amtu=amtu)

Here, each element of `pop` has the type `NCASubject`. It is a lazy data
structure and actual computations are not performed.  When we are instantiating
`NCASubject`, it only performs data checking and cleaning. To calculate AUC,
one can do:

In [ ]:
NCA.auc(pop)

`AUClast` is the area under the curve from the first observation to the last
observation. To compute `AUClast` on the second individual, one would do:

In [ ]:
NCA.auc(pop[2], auctype=:last)

Or to compute the AUC on every individual, one would do:

In [ ]:
NCA.auc(pop, auctype=:last)

One can also compute AUC on a certain interval. To compute AUC on the interval
$[10, \infty]$ on the first individual

In [ ]:
NCA.auc(pop[1], interval=(10,Inf).*timeu)

Note that we need to apply the time unit to the interval for units
compatibility. One can also specify multiple intervals

In [ ]:
NCA.auc(pop[1], interval=[(10,Inf).*timeu, (10, 15).*timeu])

In many cases, the AUC commands may need to extrapolate in order to cover the
desired interval. To see the percentage of extrapolation
($\frac{\text{extrapolated AUC}}{\text{Total AUC}}\cdot 100$), you can use
the command:

In [ ]:
NCA.auc_extrap_percent(pop[1])

Area under the first moment of the concentration (AUMC) is

$\int_{t_0}^{t_1} t\cdot\text{concentration}(t) dt.$

The interface of computing AUMC is exactly the same with AUC, and one needs to
change `auc` to `aumc` for calculating AUMC or related quantities. For
instance,

In [ ]:
NCA.aumc_extrap_percent(pop[1])
NCA.aumc(pop[1])

## Terminal Rate Constant ($\lambda z$)

The negative slope for concentration vs time in log-linear scale is the
terminal rate constant, often denoted by $\lambda z$. To compute $\lambda z$,
one can call

In [ ]:
NCA.lambdaz(pop[1])

To get the coefficient of determination ($r^2$), the adjusted coefficient of
determination ($adjr^2$), the $y$-intercept, the first time point used, and the
number of points used while computing $\lambda z$, one can do:

In [ ]:
NCA.lambdazr2(pop)
NCA.lambdazadjr2(pop)
NCA.lambdazintercept(pop)
NCA.lambdaztimefirst(pop)
NCA.lambdaznpoints(pop)

By default, $\lambda z$ calculation checks last $10$ or less data points, one
can change it by providing the keyword `threshold`, e.g.

In [ ]:
NCA.lambdaz(pop[1], threshold=2)

One can also specify the exact data points by passing their indices

In [ ]:
NCA.lambdaz(pop[1], idxs=[10, 15, 16])

You can also pass their time points

In [ ]:
NCA.lambdaz(pop[1], slopetimes=[1,2,3].*timeu)

## Simple functions

`T_max` is the time point at which the maximum concentration (`C_max`) is
observed, and they can be computed by:

In [ ]:
NCA.tmax(pop[1])
NCA.cmax(pop[1])
NCA.cmax(pop[1], interval=(20, 24).*timeu)
NCA.cmax(pop[1], interval=[(20, 24).*timeu, (10, 15).*timeu])

Note that `cmax` returns `C_max` and normalized `C_max` if `dose` is provided.
If `dose` is provided in the `NCASubject`, that `dose` will be used by all
computations where dose can be used.


`T_last` is the time of the last observed concentration value above the lower
limit of quantization (LLQ), and the corresponding concentration value is
(`C_last`). They can be computed by the command

In [ ]:
NCA.tlast(pop[1])
NCA.clast(pop[1])

The half-life can be computed by:

In [ ]:
NCA.thalf(pop[1])

One may need to interpolate or to extrapolate the concentration-time data. For
example, if you wanted to interpolate the concentration at $t=12$ using linear
interpolation, you would do:

In [ ]:
NCA.interpextrapconc(pop[1], 12timeu, method=:linear)

`method` can be `:linear`, `:linuplogdown`, or `:linlog`.

# Plots and Summary

To generate linear and log-linear plots, one can do:

In [ ]:
using Plots # load the plotting library
plot(pop)

to only generate the linear plot:

In [ ]:
plot(pop, loglinear=false)

Similarly, to generate log-linear plot:

In [ ]:
plot(pop, linear=false)

To calculate all NCA quantities, one can do

In [ ]:
report = NCAReport(pop)

The `NCAReport` object holds all quantities, and one can call
`NCA.to_dataframe` to get a `DataFrame` object.

In [ ]:
NCA.to_dataframe(report)

# Multiple dose

The interface of doing NCA with multiple doses is the same as doing single dose
NCA. To load the data with multiple doses, one can do

In [ ]:
multiple_doses_file = Pumas.example_nmtran_data("nca_test_data/dapa_IV_ORAL")
mdata = CSV.read(multiple_doses_file)

timeu = u"hr"
concu = u"mg/L"
amtu  = u"mg"
mpop = parse_ncadata(mdata, time=:TIME, conc=:COBS, amt=:AMT, formulation=:FORMULATION, occasion=:OCC,
                                     iv="IV", timeu=timeu, concu=concu, amtu=amtu)

To plot:

In [ ]:
plot(mpop)

To compute AUC and $\lambda z$:

In [ ]:
NCA.auc(mpop)

To get a summary, we need to provide a reference dose. In this example, we are
going to let the first dose be the reference dose.

In [ ]:
rep = NCAReport(mpop, ithdose=1)
NCA.to_dataframe(rep)